In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans
from haversine import haversine

In [2]:
from scipy.stats import gmean 

In [3]:
df = pd.read_csv("bank_dist_최종.csv")
df.head()

,Unnamed: 0,입력주소,Unnamed: 2,Y,X,cluster,subway_dist,bus_dist,shinhan_dist,senior_dist,traditional_market_dist,pop
0,0,부산광역시 부산진구 부전동 206-47,부전제2동,35.154269,129.060463,4,960.458993,829.169784,4859.409370,1430.614347,1578.630381,10808
1,1,부산광역시 중구 해관로 7 (동광동1가),동광동,35.099436,129.035601,3,1271.640482,372.031623,3601.804737,3441.781269,751.272332,2539
2,2,부산광역시 동구 범일로 84 (범일동),범일제2동,35.137482,129.059730,4,1027.792318,2269.243928,5792.134303,2224.840906,866.914250,7761
3,3,부산광역시 중구 구덕로 86 (남포동6가),남포동,35.097585,129.026365,3,853.276395,1086.899207,2829.549387,2579.278417,248.643743,859
4,4,부산광역시 금정구 장전로 28 (장전동),장전제2동,35.228588,129.085806,6,3114.471484,2615.242300,4445.873709,2538.840904,814.518381,25959


In [20]:
ahp = pd.read_csv("설문 응답.csv")
ahp02=ahp.to_numpy()

In [21]:
# 각 행의 기하평균을 계산하여 1차원 배열로 변환
gmean_arr = np.prod(ahp02, axis=1)**(1/ahp02.shape[1])

# 쌍대비교행렬 초기화
n = len(gmean_arr)
pairwise_matrix = np.ones((n, n))

# 쌍대비교행렬의 각 요소를 기하평균으로 계산
for i in range(n):
    for j in range(i+1, n):
        a = gmean_arr[i]
        b = gmean_arr[j]
        gmean = np.sqrt(a * b)
        pairwise_matrix[i, j] = gmean
        pairwise_matrix[j, i] = 1/gmean

print(pairwise_matrix)

[[1.         0.53831545 0.33991397 ... 0.30447676 0.41031917 0.4646334 ]
 [1.85764686 1.         0.79176532 ... 0.70922104 0.95576095 1.08227567]
 [2.94192091 1.2630005  1.         ... 0.44783061 0.6035058  0.68339227]
 ...
 [3.28432296 1.40999764 2.23298714 ... 1.         0.54058823 0.61214626]
 [2.43712716 1.04628674 1.65698491 ... 1.84983681 1.         0.82494095]
 [2.15223442 0.92397901 1.46328843 ... 1.63359653 1.21220797 1.        ]]


In [22]:
np01=pairwise_matrix
col_sums = np01.sum(axis=0)
np02 = np01 / col_sums[np.newaxis, :]
row_avg = np02.mean(axis=1) # 최종중요도(가중치)
np03 = np01.dot(row_avg[:, np.newaxis])/row_avg[:, np.newaxis]
consistency = (np03.mean(axis=0)-6)/5 # 일관성 지수

In [23]:
for i, j in zip(['자동화기기', '버스정류장', '지하철역', '전통시장', '경로당','노인인구'], row_avg):
    print('최종 중요도({}) : {}'.format(i, j))

최종 중요도(자동화기기) : 0.006984476176396388
최종 중요도(버스정류장) : 0.016131164424679673
최종 중요도(지하철역) : 0.010703644441978577
최종 중요도(전통시장) : 0.009936744237904851
최종 중요도(경로당) : 0.012802246472550448
최종 중요도(노인인구) : 0.009824317545320727


In [24]:
col_sums = np01.sum(axis=0) # 열의 합
print('열의 합 : {}'.format(col_sums))
np02 = np01 / col_sums # 정규화(np01의 값을 동일한 열에 있는 col_sumns의 값으로 나눠줍니다.)
print('정규화 : {}'.format(np02))
row_avg = np02.mean(axis=1) # 속성별 중요도, 가중치(mean(axis=1)은 행의 평균을 계산합니다.)
print('가중치 : {}'.format(row_avg))

for i, j in zip(['자동화기기', '버스정류장', '지하철역', '전통시장', '경로당','노인인구'], row_avg):
    print('최종 중요도({}) : {}'.format(i, j))

열의 합 : [145.10223316  62.60555338  96.84254172 108.28373491  80.28534621
 121.31771936  66.85892055  53.09458036  80.14865259 108.41322325
  57.01389458  99.04777627  96.56134428  95.84287991  57.81822163
 103.48713404  78.25528879  56.88459716  88.38885888  75.20226243
  62.55021869  86.42632925  63.8068581   68.36577245  66.08595599
  61.57024907  53.78478664  64.70022006  53.9553435   54.64943489
  64.28051158  57.16613851  53.04684525  60.92472258  51.80225094
  52.49504314  50.75604975  49.95266035  48.35172668  47.16981112
  52.10647764  45.63818853  52.73428776  44.21744415  41.28091941
  41.76208913  39.61278071  44.71346821  33.30160951  29.84055416
  29.92049333  37.03230429  41.25419707]
정규화 : [[0.00689169 0.00859853 0.00350997 ... 0.01017619 0.01108003 0.01126269]
 [0.01280233 0.01597302 0.0081758  ... 0.02370352 0.02580884 0.02623432]
 [0.02027481 0.02017394 0.01032604 ... 0.01496735 0.01629674 0.0165654 ]
 ...
 [0.02263454 0.02252193 0.02305792 ... 0.03342191 0.01459775 0

In [27]:
consistency[0]

10.426084039236235

In [10]:
# 쌍대비교행렬의 열별 합을 구합니다.
col_sum = np.sum(pairwise_matrix, axis=0)

# 열의 합으로 쌍대비교행렬을 나누어 열별 비율을 구합니다.
norm_matrix = pairwise_matrix / col_sum

# 열별 비율의 평균을 구하여 각 기준의 가중치를 구합니다.
weight = np.mean(norm_matrix, axis=1)

# 가중치를 합하면 1이 되도록 정규화합니다.
weight = weight / np.sum(weight)

# 결과 출력
print(weight)

[0.05163506 0.03584144 0.06023937 0.04261849 0.03991405 0.04614089
 0.02461412 0.04307033 0.04609341 0.03242155 0.02618801 0.03724909
 0.02493207 0.02407937 0.02378262 0.0272622  0.02056911 0.02429496
 0.0267697  0.01713904 0.01555568 0.01811516 0.01434038 0.01805025
 0.01548221 0.01421817 0.01270176 0.01930723 0.01296113 0.01446755
 0.015235   0.01067368 0.01039809 0.01289947 0.00908288 0.00968099
 0.00863006 0.00926617 0.00783261 0.00830603 0.00763216 0.00726488
 0.00632877 0.00677027 0.00566167 0.00530741 0.00496267 0.00457685
 0.00416542 0.00386035 0.00378609 0.00327326 0.00243459 0.00191618]


In [11]:
for i, j in zip(['자동화기기', '버스정류장', '지하철역', '전통시장', '경로당','노인인구'], weight):
    print('최종 중요도({}) : {}'.format(i, j))

최종 중요도(자동화기기) : 0.05163505652149283
최종 중요도(버스정류장) : 0.03584143989133486
최종 중요도(지하철역) : 0.0602393747500388
최종 중요도(전통시장) : 0.042618488967925264
최종 중요도(경로당) : 0.039914052854698845
최종 중요도(노인인구) : 0.046140888622025394


In [12]:
#스케일링
scaler=MinMaxScaler()
mas_scale = scaler.fit_transform(df[['subway_dist','bus_dist','shinhan_dist','senior_dist',
                                      'traditional_market_dist','pop']])

df_scale = pd.DataFrame(mas_scale)

df_scale.to_csv("스케일링.csv")